In [1]:
import sklearn
import numpy as np
import pandas as pd
from dev import DEVALGO 
from smote import SMOTEBoost
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score

In [2]:
MINORITY = 1
MAJORITY = 0
CLASS_NAME = "num"

In [3]:
def pprint_dict(d):
    for key, value in d.items():
        print(key + ": " + "{0:.2f}".format(value/100.0))

In [4]:
df = pd.read_csv("data/heart/heart.csv").sample(frac=1, random_state=0).reset_index(drop=True)
df = df.drop(["sex"], axis=1)
df.head()

,age,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,30,1,170,237,0,1,170,0,0.0,-9,-9,6,0
1,50,3,140,288,0,0,140,1,0.0,-9,-9,7,1
2,59,3,130,318,0,0,120,1,1.0,2,-9,3,0
3,37,4,130,173,0,1,184,0,0.0,-9,-9,-9,0
4,49,2,124,201,0,0,164,0,0.0,-9,-9,-9,0


In [5]:
# Outcomes distribution
df[CLASS_NAME].value_counts()

0    188
1     37
3     28
2     26
4     15
Name: num, dtype: int64

In [6]:
# Uncomment this when testing on vowel
df[CLASS_NAME].replace({1:1, 2:1, 3:1, 4:1}, inplace=True)
df.head()

,age,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,30,1,170,237,0,1,170,0,0.0,-9,-9,6,0
1,50,3,140,288,0,0,140,1,0.0,-9,-9,7,1
2,59,3,130,318,0,0,120,1,1.0,2,-9,3,0
3,37,4,130,173,0,1,184,0,0.0,-9,-9,-9,0
4,49,2,124,201,0,0,164,0,0.0,-9,-9,-9,0


In [7]:
# Outcomes distribution
df[CLASS_NAME].value_counts()

0    188
1    106
Name: num, dtype: int64

In [8]:
# Calculate imbalanced ratio
frq = df[CLASS_NAME].value_counts()
ratio = max(frq)/min(frq)
"{0:.1f}".format(ratio)

'1.8'

# Use the following for testing Decision tree

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [9]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train, y_train)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [10]:
pprint_dict(res)

precision: 0.68
recall: 0.63
f1-score: 0.65
specificity: 0.84
sensitivity: 0.63
overall accuracy: 0.77
auc: 0.74
g_mean: 0.73


# Use the following for testing SMOTE

- run 100 iterations
- 50/50 training/testing
- Decision tree
- N = 200

In [11]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    sm = SMOTE(random_state=0)
    sm.fit(X_train, y_train)
    X_train_r, y_train_r = sm.fit_resample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (147, 12)
Outcome distribution of X_train before oversampling: [92 55]
Shape of X_train after oversampling: (184, 12)
Outcome distribution of X_train after oversampling: [92 92]


In [12]:
pprint_dict(res)

precision: 0.67
recall: 0.57
f1-score: 0.62
specificity: 0.85
sensitivity: 0.57
overall accuracy: 0.76
auc: 0.71
g_mean: 0.70


# Use the following for testing ADASYN

- run 100 iterations
- 50/50 training/testing
- Decision tree
- A fully balanced dataset after synthesizing
- Dth = 0.75 (Dth is a preset threshold for the maximum tolerated degree of class imbalance ratio)

In [13]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    ada = ADASYN(random_state=0)
    ada.fit(X_train, y_train)
    X_train_r, y_train_r = ada.fit_resample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (147, 12)
Outcome distribution of X_train before oversampling: [92 55]
Shape of X_train after oversampling: (176, 12)
Outcome distribution of X_train after oversampling: [92 84]


In [14]:
pprint_dict(res)

precision: 0.68
recall: 0.75
f1-score: 0.71
specificity: 0.81
sensitivity: 0.75
overall accuracy: 0.79
auc: 0.78
g_mean: 0.78


# Use the following for testing SMOTEBoost

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [15]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = SMOTEBoost(random_state=0)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [16]:
pprint_dict(res)

precision: 0.55
recall: 0.75
f1-score: 0.63
specificity: 0.68
sensitivity: 0.75
overall accuracy: 0.70
auc: 0.71
g_mean: 0.71


# Use the following for testing Dev_algo

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [17]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    unique, counts = np.unique(y_train, return_counts=True)
    frequency = dict(zip(unique, counts))
    clf1 = DEVALGO(random_state=0, n_samples=frequency[MAJORITY])
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [18]:
 pprint_dict(res)

precision: 0.60
recall: 0.74
f1-score: 0.66
specificity: 0.74
sensitivity: 0.74
overall accuracy: 0.74
auc: 0.74
g_mean: 0.74
